# Notebook 4: Efficient Location and Time Series Access Using Polytope Feature Extraction

Notebook 4 showcases efficient geo-location and time series access using polytope feature extraction. By leveraging this feature, we significantly reduce data retrieval from storage, optimizing workflows and resource utilization.

1. **Configuring Access to Polytope**

In [ ]:
import os
import dataclasses as dc
from meteodatalab import mars

os.environ["POLYTOPE_USER_KEY"] = "XXXXXXX"
os.environ["POLYTOPE_ADDRESS"] = "https://polytope-dev.mchml.cscs.ch"
os.environ["POLYTOPE_LOG_LEVEL"] = "INFO"


2. **Selecting geo-location point of interest**

In [ ]:
zrh = mars.Point(47.47, 8.55)

3. **Querying the data** <br>
The `feature` attribute tells polytope to extract **only the relevant data at the given point**.
The amount of data that is retrieved from storage is signficantly reduced.

In [ ]:
request = mars.Request(
    param="T_2M",
    date="20250402",
    time="1200",
    model=mars.Model.ICON_CH1_EPS,
    levtype=mars.LevType.SURFACE,
    number=2,
    feature=mars.TimeseriesFeature(
        points=[zrh],
        range=mars.Range(start=0, end=30),
        time_axis="step",
    )
)

In [ ]:
from meteodatalab import mch_model_data

ds = mch_model_data.get_from_polytope(request)

4. **Visualizing the data**

In [ ]:
ds["t_2m"].plot()

5. **Plotting ensemble members of timeseries**

In [ ]:
# mars.Request is a dataclass
requests = [
    dc.replace(request, number=num, ) for num in range(1, 3)
]
requests

In [ ]:
ememb_ds = []
for r in requests:
    ememb_ds.append(mch_model_data.get_from_polytope(r))

# from concurrent.futures import ThreadPoolExecutor
# with ThreadPoolExecutor(max_workers=5) as exec:
# #   ememb_ds = list(exec.map(mch_model_data.get_from_polytope, requests))


In [ ]:

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

labels = []

plt.figure()

for mem_ds in ememb_ds:
    ds_xr = mem_ds["t_2m"]
    label = f"Member {ds_xr['number'].values[0]}"
    data = ds_xr.values.flatten()
    time = ds_xr.t.values
    plt.plot(time, data, label=label)

plt.xlabel("t")
plt.ylabel("t_2m")
plt.legend()
